# 나만의 사고 AI 서비스 '🚀 사고닷 🚀'
사고 관련 법률 상담을 AI를 활용하여 자동화하고,    
단계별 질문을 통해 사용자 맞춤형 법률 정보를 제공   


In [7]:
!pip install --upgrade jsonlines openai langchain langchain-openai langchain-community beautifulsoup4 langchain_chroma chromadb==0.5.3

Defaulting to user installation because normal site-packages is not writeable
  Attempting uninstall: openai
    Found existing installation: openai 1.65.3
    Uninstalling openai-1.65.3:
      Successfully uninstalled openai-1.65.3


In [ ]:
!pip install pymupdf sentence-transformers faiss-cpu


Defaulting to user installation because normal site-packages is not writeable
ERROR: Invalid requirement: 'pymupdf,': Expected end or semicolon (after name and no valid version specifier)
    pymupdf,
           ^


In [ ]:
import requests
from bs4 import BeautifulSoup
import os

#os.environ["OPENAI_API_KEY"] = "<OpenAI_API의 API 키>"
os.environ["OPENAI_API_KEY"] = "YOUR_KEY"

os.environ['USER_AGENT']='MyCustomAgent'
# 아래 코드 Warning 제거

/Users/now_ung/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [10]:
import fitz  # PyMuPDF 라이브러리

def extract_text_from_pdf(pdf_path):
    text = ""
    doc = fitz.open(pdf_path)  # PDF 열기
    for page in doc:
        text += page.get_text("text") + "\n"
    return text

# 저장된 PDF 파일들 처리
pdf_folder = "pdf_files"
pdf_texts = {}

for pdf_file in os.listdir(pdf_folder):
    pdf_path = os.path.join(pdf_folder, pdf_file)
    text = extract_text_from_pdf(pdf_path)
    pdf_texts[pdf_file] = text
    print(f"텍스트 추출 완료: {pdf_file}")


텍스트 추출 완료: 한글 6법전(헌법, 민법, 형법, 상법, 민사소송법, 형사소송법).pdf
텍스트 추출 완료: 한글 4법전(헌법, 민법, 형법, 형사소송법).pdf


In [11]:

from sentence_transformers import SentenceTransformer

# 사전 학습된 Transformer 모델 로드
model = SentenceTransformer("all-MiniLM-L6-v2")  # 빠르고 가벼운 모델

# PDF별 벡터 변환
pdf_embeddings = {}

for pdf_file, text in pdf_texts.items():
    embedding = model.encode(text)
    pdf_embeddings[pdf_file] = embedding
    print(f"벡터 변환 완료: {pdf_file}")


/Users/now_ung/Library/Python/3.9/lib/python/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


벡터 변환 완료: 한글 6법전(헌법, 민법, 형법, 상법, 민사소송법, 형사소송법).pdf
벡터 변환 완료: 한글 4법전(헌법, 민법, 형법, 형사소송법).pdf


In [12]:
import faiss
import numpy as np
from sentence_transformers import SentenceTransformer

# 임베딩 모델 로드 (빠르고 가벼운 모델)
model = SentenceTransformer("all-MiniLM-L6-v2")

# 벡터 변환
pdf_embeddings = {}
for pdf_file, text in pdf_texts.items():
    embedding = model.encode(text)
    pdf_embeddings[pdf_file] = embedding
    print(f"벡터 변환 완료: {pdf_file}")

# FAISS 벡터 DB 생성
embedding_dim = len(next(iter(pdf_embeddings.values())))  # 벡터 차원
index = faiss.IndexFlatL2(embedding_dim)  

# 벡터 삽입
pdf_files = list(pdf_embeddings.keys())
pdf_vectors = np.array(list(pdf_embeddings.values()), dtype="float32")
index.add(pdf_vectors)

# FAISS 인덱스 저장
faiss.write_index(index, "faiss_index.bin")
np.save("pdf_files.npy", np.array(pdf_files))  # PDF 파일명 저장

print("FAISS DB 저장 완료 ✅")


벡터 변환 완료: 한글 6법전(헌법, 민법, 형법, 상법, 민사소송법, 형사소송법).pdf
벡터 변환 완료: 한글 4법전(헌법, 민법, 형법, 형사소송법).pdf
FAISS DB 저장 완료 ✅


In [ ]:
from openai import OpenAI

# OpenAI 클라이언트 연결
client = OpenAI(api_key="YOUR_KEY")

class Agent:
    def __init__(self, system_prompt=""):
        self.system_prompt = system_prompt
        self.messages = [{"role": "system", "content": system_prompt}]  # 초기 프롬프트 포함

    def __call__(self, message):
        """사용자의 입력을 받아 AI 응답을 생성"""
        self.messages.append({"role": "user", "content": message})
        result = self.execute()
        self.messages.append({"role": "assistant", "content": result})
        return result

    def execute(self):
        """AI에게 메시지를 보내고 응답을 받음"""
        completion = client.chat.completions.create(
            model="gpt-4o-mini",
            temperature=0,
            messages=self.messages
        )
        return completion.choices[0].message.content

# **시스템 프롬프트 (단계별 상담 + 가이드라인 유지)**
prompt = """
당신은 사고 관련 법률 정보를 제공하는 AI 법률 상담사입니다.
사용자가 사고와 관련된 법률적 도움을 요청할 때, 아래의 단계별 질문 프레임워크를 따라 포괄적인 질문에서 구체적인 질문으로 진행하며 정확한 법률 정보를 제공해야 합니다.

# 시스템 지침
1. 사용자의 첫 질문이나 문의를 들은 후, 단계별로 질문을 진행하세요.
2. 각 단계마다 답변을 듣고 그에 맞는 다음 단계의 질문을 선택하세요.
3. 사용자의 응답에 따라 관련 법률 정보와 조언을 제공하세요.
4. 모든 법률 정보는 정확하고 최신의 정보여야 합니다.
5. 전문적이지만 이해하기 쉬운 언어로 소통하세요.
6. 사용자가 추가 질문이나 명확한 설명을 요청할 경우, 즉시 응답하세요.
7. 법률적 조언이 필요한 경우 실제 변호사와 상담할 것을 권장하세요.

# 응답 가이드라인
1. 사용자의 응답이 불분명하거나 부족한 경우, 추가 질문을 통해 명확히 하세요.
2. 사용자가 질문 단계를 건너뛰거나 다른 주제로 전환하려 할 경우, 필요한 정보를 얻기 위해 질문을 다시 유도하세요.
3. 사용자가 특정 법률 용어나 절차에 대해 이해하지 못하는 경우, 이해하기 쉽게 설명해주세요.
4. 유사한 판례나 법률 조항을 인용할 때는 정확한 출처와 내용을 제공하세요.
5. 모든 정보와 조언은 객관적이고 사실에 기반해야 합니다.
6. 사용자의 감정적 상태를 고려하여 공감적인 태도를 유지하세요.
7. 사용자의 상황이 긴급하거나 심각한 경우, 적절한 긴급 조치나 전문가 상담을 권장하세요.

# 질문 프로세스
1. Thought: 질문을 분석하여 어떤 정보를 더 수집해야 하는지 결정하세요.
2. Action: 필요한 정보를 얻기 위한 질문을 생성하세요.
3. 사용자 응답 후 다음 단계로 진행하세요.
"""

# AI 상담 Agent 생성
abot = Agent(system_prompt=prompt)

def interactive_law_consultation():
    """사용자가 질문하면 AI가 단계별 질문을 진행하며 정보를 수집"""
    print("\n[나만의 사고 AI 서비스 '🚀 사고닷 🚀' 법률 상담 챗봇] 상담을 시작합니다. (종료하려면 'exit' 입력)\n")

    # 초기 사용자 질문 입력
    while True:
        # 사용자 입력
        user_query = input("사용자: ")

        # "exit" 입력 시 상담 종료
        if user_query.lower() == "exit":
            print("\n[나만의 사고 AI 서비스 '🚀 사고닷 🚀' 법률 상담 챗봇] 상담을 종료합니다.")
            break

        # AI 응답 생성 및 즉시 출력
        ai_response = abot(user_query)
        print("\n🚀 사고닷 🚀 : ", ai_response)


# **실행**
interactive_law_consultation()



[나만의 사고 AI 서비스 '🚀 사고닷 🚀' 법률 상담 챗봇] 상담을 시작합니다. (종료하려면 'exit' 입력)


🚀 사고닷 🚀 :  안타까운 일이네요. 도난 사건에 대해 좀 더 구체적으로 알아보겠습니다. 

우선, 도난 사건이 발생한 날짜와 시간은 언제였나요? 그리고 도난당한 지갑 안에는 어떤 물건들이 있었는지 알려주실 수 있나요?

[나만의 사고 AI 서비스 '🚀 사고닷 🚀' 법률 상담 챗봇] 상담을 종료합니다.


In [ ]:
import os
from langchain_openai import ChatOpenAI
from langchain_community.retrievers import TavilySearchAPIRetriever
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain.schema import Document

# OpenAI API Key
os.environ["OPENAI_API_KEY"] = (
    "YOUR_KEY"
)

# Tavily API key
os.environ["TAVILY_API_KEY"] = "TAVILY KEY"

# tavily에서 받은 결과를 처리하여 판례 요약을 하나의 Document로 결합하는 부분을 구현
def create_document_from_result(result):
    # 결과를 하나의 텍스트로 결합
    full_content = ""
    
    # 각 항목을 순회하면서 내용을 합침
    for item in result:
        full_content += f"Title: {item['title']}\n"
        full_content += f"Summary: {item['content']}\n"
        full_content += f"URL: {item['url']}\n"
        full_content += "\n"  # 항목 간 구분

    # full_content를 하나의 Document로 만듬
    document = Document(page_content=full_content)
    
    return document

# tavily 호출 후 Document로 변환
result = tavily.invoke('학교 앞 교통사고 관련 실제 판례 기사 내용')

# Document 생성
document = create_document_from_result(result)

# 문서 내용 확인
print(document.page_content)

# 문서를 외부로 보내는 코드 (예시로 파일로 저장)
with open("result_document.txt", "w") as f:
    f.write(document.page_content)


def format_docs(docs):
    return "\n\n".join([d.page_content for d in docs])

template = """
    Answer the question based only on the following context:

        {context}

    Please focus on summarizing the numerical details, such as:
    - Fines
    - Lawyer fees
    - Sentences (length of prison term or probation)
    - Court trial duration

    Question: {question}
"""
prompt = ChatPromptTemplate.from_template(template)
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)
retriever = TavilySearchAPIRetriever(
    k=3, search_depth="advanced", include_domains=["news"]
)

chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
)

result = chain.invoke("학교 앞 교통사고 관련 형량이나 벌금은 얼마나 되나요?")

print(result.content)